In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
import cv2
import torch
from ultralytics import YOLO

def check(s):
    if len(s) == 6 and s[0].isalpha() and s[1].isalpha():
        return 0
    if "NN" in s and len(s) in {8, 9, 10}:
        return 0
    if len(s) not in {8, 9}:
        return 1
    if not s[2].isalpha():
        return 1
    if any(not c.isdigit() for i, c in enumerate(s) if i != 2 and i != 3):
        return 1
    return 0

plate_detector = YOLO("/content/drive/MyDrive/Colab Notebooks/Stage 1/run_l/exp5/weights/best.pt")
char_recognizer = YOLO("/content/drive/MyDrive/Colab Notebooks/Stage 2/run_l/exp2/weights/best.pt")

def display_plate(frame, plate_text, x1, y1, x2, y2):
    cv2.rectangle(frame, (x1, y1), (x2, y2), (16, 12, 23), 2)
    text_size = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
    cv2.rectangle(frame, (x1, y1 - text_size[1] - 10), (x1 + text_size[0], y1), (16, 12, 23), -1)
    cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (175,154,247), 2)

def recognize_plate_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) )

    plates_dict1 = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        plates_dict2 = {}

        plate_results = plate_detector(frame)

        for result in plate_results:
            for box in result.boxes.xyxy:
                x1, y1, x2, y2 = map(int, box)
                plate_region = frame[y1:y2, x1:x2]

                conf = 0
                plate_text = ''

                if plate_text == '':
                    char_dict = {}
                    char_results = char_recognizer(plate_region)

                    for char_result in char_results:
                        for idx, char_box in enumerate(char_result.boxes.xyxy):
                            cx, cy, w, h = char_box
                            cls_index = int(char_result.boxes.cls[idx])
                            conf += char_result.boxes.conf[idx].item()

                            if cls_index < 9:
                                char_label = chr(cls_index + 49)
                            elif cls_index < 17:
                                char_label = chr(cls_index + 56)
                            elif cls_index < 21:
                                char_label = chr(cls_index + 58)
                            elif cls_index == 21:
                                char_label = chr(cls_index + 59)
                            elif cls_index < 26:
                                char_label = chr(cls_index + 61)
                            elif cls_index < 29:
                                char_label = chr(cls_index + 62)
                            else:
                                char_label = '0'

                            char_dict[int(idx)] = (char_label, cx, cy, w, h)

                    char_items = sorted(char_dict.items(), key=lambda item: item[1][1])

                    i = 0
                    front = 0
                    flip = 0

                    while i < len(char_items) - 1:
                        idx1, (char1, cx1, cy1, mx1, my1) = char_items[i]
                        idx2, (char2, cx2, cy2, mx2, my2) = char_items[i + 1]

                        print(char1)
                        print(char_items)
                        print('front =', front)

                        if (my1) - (cy2) < (3 / 10) * (my1 - cy1):
                            if flip == char_items[i + 1]:
                                break
                            if flip == 0:
                                flip = char_items[i + 1]
                            char_items.append(char_items.pop(i + 1))
                            front = i+1
                            i-=1
                        elif (my2) - (cy1) < (3 / 10) * (my2 - cy2):
                            char_items.insert(front, char_items.pop(i + 1))
                            #char_items.append(char_items.pop(i+1))
                            i = front - 1
                            front += 1
                        i += 1

                    plate_text = "".join([char for _, (char, _, _, _, _) in char_items])

                    if conf != 0:
                      conf = conf/len(plate_text)

                if check(plate_text) == 0:
                    oiou = 0.4
                    for plate, (px1, py1, px2, py2, cf) in plates_dict1.items():
                        ix1, iy1 = max(px1, x1), max(py1, y1)
                        ix2, iy2 = min(px2, x2), min(py2, y2)

                        intersection_area = max(0, ix2 - ix1) * max(0, iy2 - iy1)
                        box_area = (x2 - x1) * (y2 - y1)
                        plate_area = (px2 - px1) * (py2 - py1)
                        iou = intersection_area / float(box_area + plate_area - intersection_area)

                        if iou >= oiou and cf > conf:
                            plate_text = plate
                            conf = cf
                            oiou = iou
                            break
                    plates_dict2[plate_text] = (x1, y1, x2, y2, conf)
                    display_plate(frame, plate_text, x1, y1, x2, y2)
                else:
                    oiou = 0.4
                    conf = 0
                    for plate, (px1, py1, px2, py2, cf) in plates_dict1.items():
                        ix1, iy1 = max(px1, x1), max(py1, y1)
                        ix2, iy2 = min(px2, x2), min(py2, y2)

                        intersection_area = max(0, ix2 - ix1) * max(0, iy2 - iy1)
                        box_area = (x2 - x1) * (y2 - y1)
                        plate_area = (px2 - px1) * (py2 - py1)
                        iou = intersection_area / float(box_area + plate_area - intersection_area)

                        if iou >= oiou and cf > conf:
                            plate_text = plate
                            conf = cf
                            oiou = iou
                            break

                    if check(plate_text) == 0:
                        plates_dict2[plate_text] = (x1, y1, x2, y2, conf)
                        display_plate(frame, plate_text, x1, y1, x2, y2)
                    elif plate_text != '':
                        display_plate(frame, 'UNKNOWN', x1, y1, x2, y2)

        plates_dict1 = plates_dict2
        out.write(frame)

    cap.release()
    out.release()

# Paths
input_path = "/content/drive/MyDrive/Colab Notebooks/Test_Vid/Test1.mp4"
output_path = "/content/drive/MyDrive/Colab Notebooks/Output_Vid/Output1.mp4"

recognize_plate_video(input_path, output_path)


0: 384x640 1 0, 17.6ms
Speed: 2.0ms preprocess, 17.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 0, 1 3, 1 4, 1 20, 1 29, 12.1ms
Speed: 1.7ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)

0: 384x640 (no detections), 12.5ms
Speed: 1.8ms preprocess, 12.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.8ms
Speed: 1.8ms preprocess, 11.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.7ms
Speed: 1.8ms preprocess, 12.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.1ms
Speed: 1.8ms preprocess, 12.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.8ms
Speed: 1.8ms preprocess, 11.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.0ms
Speed: 1.9ms preprocess, 12.0ms inf